In [ ]:
import random

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.signal as signal

np.random.seed(1337)

# Preprocessing

This notebooks explore the preprocessing steps for time series data ultimately implemented as part of the `data_pipeline.py` script. The preprocessing steps include:

1. **Time Series Preprocessing**: This includes converting the time column to a datetime object and setting it as the index.
2. **Crop**: This involves cropping the data to remove the first and last few seconds of the recording.
3. **Resample**: This involves resampling the data to a lower frequency.
4. **Segmentation**: This involves segmenting the data into smaller chunks.
5. **Feature Extraction**: This involves extracting features from the data.
6. **Smoothing**: This involves applying filters to smooth the data.

Let's start by loading the data and visualizing it.

## Load Data

For the purpose of this notebook, we will load a random file from the dataset.

In [ ]:
data = pd.read_parquet("../data/cache/raw_data_db_cache.parquet")
data = data[data['file_hash'] == data['file_hash'].sample(1).values[0]] # select a random file

data.drop(columns=['recording_time', 'result', 'table'], inplace=True)
data.columns = data.columns.str.replace('_time', 'time')
data = data.rename(columns={'accelerometer_x': 'x', 'accelerometer_y': 'y', 'accelerometer_z': 'z'})

data = data.sort_values('time')
data = data.reset_index(drop=True)

sensor_data = data.copy()
data.head()

Afterwards we can define a function to visualize the data.

In [ ]:
def visualize_data(data, title, first_n_seconds=None):
    if first_n_seconds is not None:
        data = data[data.index <= data.index[0] + pd.Timedelta(seconds=first_n_seconds)]
    
    plt.figure(figsize=(10, 6))
    plt.plot(data.index, data['x'], label='X-axis')
    plt.plot(data.index, data['y'], label='Y-axis')
    plt.plot(data.index, data['z'], label='Z-axis')
    plt.legend()
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Acceleration')
    plt.show()

## 1. Time Series Preprocessing

The first step is to preprocess the time series data by converting the time column to a datetime object and setting it as the index. Afterwards we ensure the data is sorted by time.

In [ ]:
sensor_data['time'] = pd.to_datetime(sensor_data['time'], unit='ns')
sensor_data = sensor_data.set_index('time', drop=True)
sensor_data = sensor_data.sort_index()
visualize_data(sensor_data.copy(), 'Preprocessed Data', first_n_seconds=5)

As we can already see on the visualised data, the data is quite noisy in the first few seconds. This comes from the measurement device being moved into position as part of the measurement concept. This is also the case for the last few seconds of the recording. We can remove this noise by cropping the data.

## 2. Crop

The next step is to crop the data to remove the first and last few seconds of the recording. This can be done by specifying the start and end time to crop the data.

In [ ]:
start_crop = pd.Timedelta(seconds=5)
end_crop = pd.Timedelta(seconds=5)

cropped_data = sensor_data[sensor_data.index.min() + start_crop:sensor_data.index.max() - end_crop]
visualize_data(cropped_data.copy(), 'Cropped Data', first_n_seconds=5)

We can see the time axis has been shifted to start at +5 seconds. This should remove the noise as stated in the measurement concept. Next we can resample the data to a lower frequency as the original data could be sampled at a higher frequency than needed leading to an possible unnecessary high computational cost or not standardized data.

## Resample

Resampling the data to a lower frequency can help reduce the computational cost and standardize the data. For example, if the original data was sampled at 100 Hz, we can resample it to 50 Hz. This can be done using the `resample` method in pandas.

In [ ]:
def visualize_resampling(original_data, resampled_data):
    plt.figure(figsize=(15, 7))
    
    plt.plot(original_data.index, original_data['x'], label='Original Data (X-axis)', alpha=0.5, linestyle='-', marker='o', markersize=4)
    plt.plot(original_data.index, original_data['y'], label='Original Data (Y-axis)', alpha=0.5, linestyle='-', marker='o', markersize=4)
    plt.plot(original_data.index, original_data['z'], label='Original Data (Z-axis)', alpha=0.5, linestyle='-', marker='o', markersize=4)
    
    plt.plot(resampled_data.index, resampled_data['x'], label='Resampled Data (X-axis)', linestyle='-', marker='x', markersize=7)
    plt.plot(resampled_data.index, resampled_data['y'], label='Resampled Data (Y-axis)', linestyle='-', marker='x', markersize=7)
    plt.plot(resampled_data.index, resampled_data['z'], label='Resampled Data (Z-axis)', linestyle='-', marker='x', markersize=7)
    
    plt.legend()
    plt.title('Comparison of Original and Resampled Data')
    plt.xlabel('Time')
    plt.ylabel('Values')
    plt.tight_layout()
    plt.show()

numeric_columns = cropped_data.select_dtypes(include='number').columns
cropped_data_numeric = cropped_data[numeric_columns]

rate = f"{int(1E6 / 50)}us"
resampled_data = cropped_data_numeric.resample(rate).mean()  # 100 Hz => 50 Hz == 1E6 / 50 us 

start_time = cropped_data_numeric.index[0]
end_time = start_time + pd.Timedelta(seconds=1)
cropped_data_subset = cropped_data_numeric[start_time:end_time]
resampled_data_subset = resampled_data[start_time:end_time]

visualize_resampling(cropped_data_subset, resampled_data_subset)

The resampled data is now at 50 Hz. We can see the data has been downsampled and the values have been averaged over the new time intervals. We can also notice that the lines do not overlap perfectly due to the averaging process. Overall, the characteristics of the data are preserved.

## 4. Segmentation

Segmentation involves dividing the data into smaller chunks. This is a necessary step for model training as the models are trained ona fixed context window. This can be done by specifying the segment size and overlap between segments. 

In [ ]:
segment_size = pd.Timedelta(seconds=5)
overlap = pd.Timedelta(seconds=2)

start_time = resampled_data.index.min()
segments = []

while start_time + segment_size <= resampled_data.index.max():
    end_time = start_time + segment_size
    segments.append(resampled_data[start_time:end_time])
    start_time = end_time - overlap
    
print(f'Number of segments: {len(segments)}')
visualize_data(segments[0], 'First Segment')

Overlap essentially means that the segments will have some overlapping data points. For example, if the segment size is 5 seconds and the overlap is 2 seconds, then the first segment will contain data points from 0 to 5 seconds, the second segment will contain data points from 3 to 8 seconds, and so on. This can be done by iterating over the data and creating segments of the specified size and overlap.

In [ ]:
def visualize_segment_overlap(segments, column='x'):    
    plt.figure(figsize=(15, 10))
    colors = plt.cm.jet(np.linspace(0, 1, len(segments)))
    
    for i, segment in enumerate(segments):
        plt.plot(segment.index, segment[column],
                 label=f'Segment {i+1}', color=colors[i], alpha=0.7)
    
    plt.title(f'Visualization of Segment Overlap for Accelerometer {column.upper()}-axis')
    plt.xlabel('Time')
    plt.ylabel('Value (with offset for visualization)')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
visualize_segment_overlap(segments[:3])

We can see that the segments overlap by 2 seconds. This can server as a form of data augmentation as the model will see similar data points in different segments. This can help the model generalize better. Next we can extract features from the data.


# 5. Feature Extraction

Feature extraction involves extracting features from the data.

## Frequency Domain

The frequency domain can provide insights into the underlying patterns in the data. We can visualize the frequency domain using the Fast Fourier Transform (FFT).

In [ ]:
def visualize_frequency_domain(data, sampling_rate, title='Frequency Domain'):
    fft_result = np.fft.fft(data)
    frequencies = np.fft.fftfreq(data.shape[0], 1/sampling_rate)
    frequencies = np.fft.fftshift(frequencies)
    fft_result = np.fft.fftshift(fft_result)
    
    plt.figure(figsize=(15, 5))
    plt.plot(frequencies, np.abs(fft_result))
    plt.title(title)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')
    plt.xlim(0, np.max(frequencies) / 2)
    plt.show()

visualize_frequency_domain(segments[0]['x'], 1E6 / 50, title='Frequency Domain for X-axis (First Segment)')

We can see the frequency domain for the first segment of the X-axis data. We can see that the frequency domain provides insights into the underlying patterns in the data. We can also calculate the correlation between the different axes of the accelerometer data. There are some quite some high frequencies on the upper end of the spectrum. This can be due to the noise in the data.

## Correlation

We can calculate the correlation between the different axes of the accelerometer data. This can help us understand how the different axes are related to each other. We can visualize the correlation using a heatmap.

In [ ]:
def visualize_correlation_heatmap(data, title='Correlation Heatmap'):
    # rename xyz columns to accelerometer_x, accelerometer_y, accelerometer_z
    data = data.copy().rename(columns={'x': 'accelerometer_x', 'y': 'accelerometer_y', 'z': 'accelerometer_z'})
    
    correlation_matrix = data.corr()
    plt.figure(figsize=(16, 12))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', 
                fmt='.2f', cbar=True, square=True, annot_kws={'size': 10}, 
                mask=np.triu(correlation_matrix, k=1))
    plt.title(title)
    plt.show()
    
visualize_correlation_heatmap(segments[0].copy(), title='Correlation Heatmap for First Segment')

The correlation heatmap provides insights into expected and unexpected relationships among the sensor data axes. Strong correlations among orientation data axes are anticipated, as orientation components (quaternions and angles) typically have interdependencies due to their mathematical and physical relationships in representing 3D orientation. Additionally, the negative correlation between gravity axes, especially between gravity_x and gravity_z, is expected due to their orthogonal nature in a fixed reference frame, where an increase in one axis component often results in a decrease in another.

On the other hand, weak correlations among gyroscope axes are surprising, as rotational movements usually involve interactions across multiple axes. This suggests that the gyroscope data might exhibit more complex dynamics or noise, leading to seemingly independent axis movements. These statements correctly reflect the relationships observed in the heatmap, providing a basis for further analysis and interpretation of the sensor data.

# 6. Smoothing

Smoothing involves applying filters to smooth the data. This can help remove noise and make the underlying patterns more visible. We can apply filters such as the Butterworth filter and the moving average filter.

## Butterworth Filter

The Butterworth filter is a type of signal processing filter designed to have a frequency response as flat as possible in the passband. It is often used in digital signal processing to reduce noise and improve the signal-to-noise ratio. We can apply the Butterworth filter to the accelerometer data.

In [ ]:
def _calc_butterworth_filter(order, cutoff, sampling_rate):
    nyquist = 0.5 * sampling_rate
    normal_cutoff = cutoff / nyquist
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    return b, a


def apply_butterworth_filter(data, order, cutoff, sampling_rate):
    b, a = _calc_butterworth_filter(order, cutoff, sampling_rate)
    return signal.filtfilt(b, a, data)


def visualize_butterworth_filter(data, filtered_data, title='Butterworth Filter'):
    plt.figure(figsize=(15, 5))
    plt.plot(data.index, data, label='Original Data', alpha=0.5)
    plt.plot(data.index, filtered_data, label='Filtered Data', linestyle='--')
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Values')
    plt.legend()
    plt.show()
    
    
butterworth_segment = segments.copy()[0]
cutoff, order = 6, 4
sampling_rate = 50
for column in ['x', 'y', 'z']:
    filtered_data = apply_butterworth_filter(butterworth_segment[column], order, cutoff, sampling_rate)
    visualize_butterworth_filter(butterworth_segment[column], filtered_data, title=f'Butterworth Filter Application ({column}-axis, Order={order}, Cutoff={cutoff} Hz)')

As we can see, the Butterworth filter has been applied to the accelerometer data. The filter has removed some of the high-frequency noise from the data, making the underlying patterns more visible. We can also apply the moving average filter to the data. There is certainly a trade-off between the amount of noise removed and the amount of signal lost. The Butterworth filter is a low-pass filter, which means it removes high-frequency noise from the data. The cutoff frequency determines the frequency above which the filter starts to remove noise. The order of the filter determines the sharpness of the cutoff. A higher order filter will have a steeper cutoff but may introduce more distortion to the signal.

## Moving Average

The moving average filter is a simple low-pass filter that averages the data over a window of a specified size. This can help smooth the data and remove noise. We can apply the moving average filter to the accelerometer data.

In [ ]:
moving_avg_segment = random.choice(segments).copy()

def visualize_moving_average(data, filtered_data, title='Moving Average'):
    plt.figure(figsize=(15, 5))
    plt.plot(data.index, data, label='Original Data', alpha=0.5)
    plt.plot(data.index, filtered_data, label='Filtered Data', linestyle='--')
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Values')
    plt.legend()
    plt.show()

def apply_moving_average(segment, window_size_s, sampling_rate, columns=['x', 'y', 'z']):
    window_size = int(window_size_s * sampling_rate)
    for column in columns:
        original_data = segment[column].copy()
        rolling_avg = segment[column].rolling(window=window_size).mean()
        rolling_avg.iloc[:window_size] = rolling_avg.iloc[window_size]
        segment.loc[:, column] = rolling_avg
        visualize_moving_average(original_data, rolling_avg, 
                                 title=f'Moving Average Application ({column}-axis, Window Size={window_size_s} s)')

apply_moving_average(moving_avg_segment.copy(), 0.2, sampling_rate)

As well as the butterworth filter, the moving average filter has been applied to the accelerometer data. The filter has removed some of the high-frequency noise from the data, making the underlying patterns more visible. Overall the filtering seems rather aggressive and might remove key characteristics of the data. It is important to carefully choose the filter parameters to avoid removing important information from the data.

In this case: The smaller the window size, the less noise is removed.

In [ ]:
apply_moving_average(moving_avg_segment.copy(), .1, sampling_rate)